In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_dev")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

In [0]:
df_aisles = spark.table(f"{catalogo}.{esquema_source}.aisles_transformed")
df_departments = spark.table(f"{catalogo}.{esquema_source}.departments_transformed")
df_products = spark.table(f"{catalogo}.{esquema_source}.products_transformed")


In [0]:
dim_product = (
    df_products.alias("p")
    .join(df_aisles.alias("a"), "aisle_id", "left")
    .join(df_departments.alias("d"), "department_id", "left")
    .select(
        col("p.product_id"),
        col("p.product_name"),
        col("p.aisle_id"),
        col("a.aisle").alias("aisle_name"),
        col("p.department_id"),
        col("d.department").alias("department_name")
    )
)
#display(dim_product)

In [0]:
dim_product.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.dim_product")